In [1]:
# all the necessary imports
import sys
sys.path.insert(0, '../scripts')
from readparquet import *
from pyspark.sql.functions import col

In [2]:
# let's import all the green taxi data for the past 6 months
sdf_g1 = read_parquet('../data/raw/green_tripdata_2021-11.parquet')
sdf_g2 = read_parquet('../data/raw/green_tripdata_2021-12.parquet')
sdf_g3 = read_parquet('../data/raw/green_tripdata_2022-01.parquet')
sdf_g4 = read_parquet('../data/raw/green_tripdata_2022-02.parquet')
sdf_g5 = read_parquet('../data/raw/green_tripdata_2022-03.parquet')
sdf_g6 = read_parquet('../data/raw/green_tripdata_2022-04.parquet')

22/07/30 12:29:34 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/07/30 12:29:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/30 12:29:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# to see what features this data have, and how this data *roughly* looks like
df_g6 = sdf_g6.sample(0.01, seed=0).toPandas()
df_g6.info()
df_g6.sample(n=6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               731 non-null    int64         
 1   lpep_pickup_datetime   731 non-null    datetime64[ns]
 2   lpep_dropoff_datetime  731 non-null    datetime64[ns]
 3   store_and_fwd_flag     644 non-null    object        
 4   RatecodeID             644 non-null    float64       
 5   PULocationID           731 non-null    int64         
 6   DOLocationID           731 non-null    int64         
 7   passenger_count        644 non-null    float64       
 8   trip_distance          731 non-null    float64       
 9   fare_amount            731 non-null    float64       
 10  extra                  731 non-null    float64       
 11  mta_tax                731 non-null    float64       
 12  tip_amount             731 non-null    float64       
 13  tolls

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
669,2,2022-04-08 07:07:00,2022-04-08 07:23:00,None,NaN,74,116,NaN,3.06,14.37,0.0,0.0,3.26,0.0,NaN,0.3,17.93,NaN,NaN,NaN
169,2,2022-04-09 05:32:41,2022-04-09 05:39:48,N,1.0,74,43,1.0,1.54,7.00,1.0,0.5,1.76,0.0,NaN,0.3,10.56,1.0,1.0,0.0
588,2,2022-04-29 09:37:18,2022-04-29 09:45:29,N,1.0,130,10,1.0,2.16,8.50,0.5,0.5,0.00,0.0,NaN,0.3,9.80,2.0,1.0,0.0
648,2,2022-04-02 12:15:00,2022-04-02 12:35:00,None,NaN,145,198,NaN,4.99,20.96,0.0,0.0,5.08,0.0,NaN,0.3,26.34,NaN,NaN,NaN
715,2,2022-04-24 20:36:00,2022-04-24 20:59:00,None,NaN,152,100,NaN,6.28,23.92,0.0,0.0,6.06,0.0,NaN,0.3,33.03,NaN,NaN,NaN
522,2,2022-04-26 04:25:39,2022-04-26 04:50:08,N,1.0,74,243,1.0,4.02,18.00,1.0,0.5,4.95,0.0,NaN,0.3,24.75,1.0,1.0,0.0


In [4]:
# all the necessary features (I ditch the ones that I absolutely sure aren't needed, and keep the ones I'm 
# still somewhat unsure whether they are needed or not, and do some feature engineering later)
feats = ["trip_type", "trip_distance", "PULocationID", "DOLocationID", "RateCodeID", "fare_amount", "extra", \
    "tip_amount", "tolls_amount", "congestion_surcharge", "total_amount"]
# mta_tax is excluded because all rides charge the same $0.5. The same reason applies for improvement_surcharge.

sdf_g1 = sdf_g1.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))
sdf_g2 = sdf_g2.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))
sdf_g3 = sdf_g3.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))
sdf_g4 = sdf_g4.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))
sdf_g5 = sdf_g5.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))
sdf_g6 = sdf_g6.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]))

In [5]:
sdf_g = sdf_g1.union(sdf_g2).union(sdf_g3).union(sdf_g4).union(sdf_g5).union(sdf_g6)
# save this as a new parquet file
sdf_g.write.mode("overwrite").parquet("../data/curated/green")
# to copy what tute1 said: "Your directory might look a bit funky. Don't worry, just leave it as is,
# (we don't have time to cover everything about Spark unfortunately) and you can just read in the directory as is."
# I TAKE YOUR WORD FOR THIS, OKAY!!

Now, do the same thing (yes, same exact thing) for the yellow taxi data.

In [6]:
sdf_y1 = read_parquet('../data/raw/yellow_tripdata_2021-11.parquet')
sdf_y2 = read_parquet('../data/raw/yellow_tripdata_2021-12.parquet')
sdf_y3 = read_parquet('../data/raw/yellow_tripdata_2022-01.parquet')
sdf_y4 = read_parquet('../data/raw/yellow_tripdata_2022-02.parquet')
sdf_y5 = read_parquet('../data/raw/yellow_tripdata_2022-03.parquet')
sdf_y6 = read_parquet('../data/raw/yellow_tripdata_2022-04.parquet')
df_y6 = sdf_y6.sample(0.01, seed=0).toPandas()
df_y6.info()
df_y6.sample(n=6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35795 entries, 0 to 35794
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               35795 non-null  int64         
 1   tpep_pickup_datetime   35795 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  35795 non-null  datetime64[ns]
 3   passenger_count        34612 non-null  float64       
 4   trip_distance          35795 non-null  float64       
 5   RatecodeID             34612 non-null  float64       
 6   store_and_fwd_flag     34612 non-null  object        
 7   PULocationID           35795 non-null  int64         
 8   DOLocationID           35795 non-null  int64         
 9   payment_type           35795 non-null  int64         
 10  fare_amount            35795 non-null  float64       
 11  extra                  35795 non-null  float64       
 12  mta_tax                35795 non-null  float64       
 13  t

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
12723,2,2022-04-12 07:08:16,2022-04-12 07:16:41,1.0,1.39,1.0,N,161,229,1,7.0,0.5,0.5,2.16,0.0,0.3,12.96,2.5,0.0
30968,2,2022-04-28 10:34:01,2022-04-28 10:37:52,1.0,1.31,1.0,N,42,42,2,5.5,0.5,0.5,0.00,0.0,0.3,6.80,0.0,0.0
21497,2,2022-04-20 06:31:20,2022-04-20 06:34:43,1.0,0.72,1.0,N,90,249,1,4.5,0.5,0.5,0.83,0.0,0.3,9.13,2.5,0.0
17938,2,2022-04-16 23:31:50,2022-04-16 23:52:45,1.0,4.06,1.0,N,162,4,1,17.0,0.0,0.5,4.06,0.0,0.3,24.36,2.5,0.0
17182,2,2022-04-16 04:32:36,2022-04-16 04:38:32,1.0,0.95,1.0,N,142,142,1,6.0,1.0,0.5,2.58,0.0,0.3,12.88,2.5,0.0
25270,2,2022-04-23 08:10:15,2022-04-23 08:12:34,1.0,0.74,1.0,N,236,263,1,4.0,0.5,0.5,1.50,0.0,0.3,9.30,2.5,0.0


In [7]:
# the yellow taxi data does not have "trip_type", so it will have one less column compared to green
# but it has "airport_fee", so the total number of columns actually ends up being the same anyway
sdf_y1 = sdf_y1.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))
sdf_y2 = sdf_y2.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))
sdf_y3 = sdf_y3.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))
sdf_y4 = sdf_y4.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))
sdf_y5 = sdf_y5.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))
sdf_y6 = sdf_y6.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col("airport_fee"), col(feats[10]))

sdf_y = sdf_y1.union(sdf_y2).union(sdf_y3).union(sdf_y4).union(sdf_y5).union(sdf_y6)
sdf_y.write.mode("overwrite").parquet("../data/curated/yellow")

This is getting repetitive, but, I still need to do it again for HVFV vehicles. (RIP my storage space)

In [8]:
sdf_fhv1 = read_parquet('../data/raw/fhvhv_tripdata_2021-11.parquet')
sdf_fhv2 = read_parquet('../data/raw/fhvhv_tripdata_2021-12.parquet')
sdf_fhv3 = read_parquet('../data/raw/fhvhv_tripdata_2022-01.parquet')
sdf_fhv4 = read_parquet('../data/raw/fhvhv_tripdata_2022-02.parquet')
sdf_fhv5 = read_parquet('../data/raw/fhvhv_tripdata_2022-03.parquet')
sdf_fhv6 = read_parquet('../data/raw/fhvhv_tripdata_2022-04.parquet')
df_fhv6 = sdf_fhv6.sample(0.001, seed=0).toPandas()  # 0.001 and it's still massive
df_fhv6.info()
df_fhv6.sample(n=6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17762 entries, 0 to 17761
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     17762 non-null  object        
 1   dispatching_base_num  17762 non-null  object        
 2   originating_base_num  12989 non-null  object        
 3   request_datetime      17762 non-null  datetime64[ns]
 4   on_scene_datetime     12989 non-null  datetime64[ns]
 5   pickup_datetime       17762 non-null  datetime64[ns]
 6   dropoff_datetime      17762 non-null  datetime64[ns]
 7   PULocationID          17762 non-null  int64         
 8   DOLocationID          17762 non-null  int64         
 9   trip_miles            17762 non-null  float64       
 10  trip_time             17762 non-null  int64         
 11  base_passenger_fare   17762 non-null  float64       
 12  tolls                 17762 non-null  float64       
 13  bcf             

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
4707,HV0003,B03404,B03404,2022-04-09 06:46:39,2022-04-09 06:49:17,2022-04-09 06:50:53,2022-04-09 07:10:06,90,262,4.180,...,1.88,2.75,0.0,0.0,17.77,N,N,,N,N
4397,HV0003,B03404,B03404,2022-04-08 20:15:47,2022-04-08 20:18:09,2022-04-08 20:19:13,2022-04-08 20:35:09,188,225,3.390,...,1.70,0.00,0.0,1.0,12.37,N,N,,N,N
1197,HV0003,B03404,B03404,2022-04-03 06:11:57,2022-04-03 06:19:34,2022-04-03 06:19:41,2022-04-03 06:25:39,144,114,0.670,...,1.63,2.75,0.0,9.3,15.87,N,N,,N,N
5077,HV0003,B03404,B03404,2022-04-09 20:16:26,2022-04-09 20:17:43,2022-04-09 20:19:46,2022-04-09 20:41:51,83,92,4.920,...,1.73,0.00,0.0,0.0,18.13,N,N,,N,N
12292,HV0003,B03404,B03404,2022-04-22 08:31:50,2022-04-22 08:35:32,2022-04-22 08:36:36,2022-04-22 08:59:04,142,148,5.820,...,3.61,2.75,0.0,5.0,37.37,N,N,,N,N
10613,HV0005,B03406,None,2022-04-19 04:54:48,NaT,2022-04-19 05:13:13,2022-04-19 05:26:00,37,198,2.244,...,1.08,0.00,0.0,0.0,9.42,N,N,N,N,N


In [9]:
sdf_fhv1.where(sdf_fhv1.shared_match_flag != "N").where(sdf_fhv1.hvfhs_license_num != "HV0005")

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag


A note on shared rides: "shared_request_flag" always shows "N". I don't know if this is an error from whoever originated the data or it's the way it is.

"shared_match_flag", however, can show "Y" even if the request is "N". The interesting thing is, only Lyft (HV0005) can show "Y", while the rest of the license num always shows "N".

In [10]:
sdf_fhv1.where(sdf_fhv1.shared_match_flag != "N").where(sdf_fhv1.hvfhs_license_num == "HV0005")

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0005,B03406,null,2021-11-01 12:40:37,null,2021-11-01 12:44:51,2021-11-01 13:09:46,79,235,12.78,1495,53.74,0.0,1.61,4.77,2.75,0.0,0.0,26.7,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 13:11:54,null,2021-11-01 13:14:08,2021-11-01 13:25:58,48,163,2.215,710,15.7,0.0,0.47,1.39,2.75,0.0,0.0,8.41,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 14:12:36,null,2021-11-01 14:17:09,2021-11-01 15:01:02,158,49,10.089,2658,121.24,0.0,3.64,10.76,2.75,0.0,0.0,47.53,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:09:31,null,2021-11-01 15:14:12,2021-11-01 15:35:05,162,265,6.742,1253,52.15,20.0,2.16,0.0,0.0,0.0,0.0,27.73,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:01:14,null,2021-11-01 15:06:33,2021-11-01 15:33:44,114,155,10.921,1631,89.63,2.17,2.75,8.15,2.75,0.0,0.0,40.09,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 14:54:38,null,2021-11-01 15:18:59,2021-11-01 15:40:15,148,262,6.458,1526,25.31,0.0,0.76,2.25,2.75,0.0,0.0,21.47,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:34:14,null,2021-11-01 15:37:54,2021-11-01 15:47:10,246,164,1.282,556,33.69,0.0,1.01,2.99,2.75,0.0,0.0,19.92,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:50:47,null,2021-11-01 15:53:34,2021-11-01 16:08:18,114,255,3.454,884,39.77,0.0,1.19,3.53,2.75,0.0,0.0,18.46,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:24:16,null,2021-11-01 15:32:17,2021-11-01 15:38:19,241,127,1.623,436,20.67,0.0,0.62,1.83,0.0,0.0,0.0,10.45,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:09:35,null,2021-11-01 15:14:53,2021-11-01 15:32:53,198,33,5.688,1111,44.14,0.0,1.32,3.92,0.0,0.0,0.0,24.01,N,Y,N,N,N


I know I only showed about 1/6 of the data here, but I tested on all of them and the result is consistent. As a result, I will ignore the "shared ride" columns due to the inconsistency of the data. Since I ignore said columns, I will also not include "Lyft" as part of my curated data, since they are the ones who have differing values in terms of shared rides.

In [11]:
# Sadly, I can't merge all these together because my device can't handle the size limit. Gonna have to do this
# the tedious way.

feats_fhv = ["hvfhs_license_num", "trip_miles", "trip_time", "PULocationID", "DOLocationID", "base_passenger_fare", \
    "tolls", "bcf", "sales_tax", "congestion_surcharge", "airport_fee", "tips", "driver_pay"]

sdf_fhv1 = sdf_fhv1.where(sdf_fhv1.hvfhs_license_num != "HV0005")
sdf_fhv2 = sdf_fhv2.where(sdf_fhv2.hvfhs_license_num != "HV0005")
sdf_fhv3 = sdf_fhv3.where(sdf_fhv3.hvfhs_license_num != "HV0005")
sdf_fhv4 = sdf_fhv4.where(sdf_fhv4.hvfhs_license_num != "HV0005")
sdf_fhv5 = sdf_fhv5.where(sdf_fhv5.hvfhs_license_num != "HV0005")
sdf_fhv6 = sdf_fhv6.where(sdf_fhv6.hvfhs_license_num != "HV0005")

sdf_fhv1 = sdf_fhv1.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv2 = sdf_fhv2.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv3 = sdf_fhv3.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv4 = sdf_fhv4.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv5 = sdf_fhv5.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv6 = sdf_fhv6.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))


In [12]:
sdf_fhv1.write.mode("overwrite").parquet("../data/curated/fhvhv1")
sdf_fhv2.write.mode("overwrite").parquet("../data/curated/fhvhv2")
sdf_fhv3.write.mode("overwrite").parquet("../data/curated/fhvhv3")
sdf_fhv4.write.mode("overwrite").parquet("../data/curated/fhvhv4")
sdf_fhv5.write.mode("overwrite").parquet("../data/curated/fhvhv5")
sdf_fhv6.write.mode("overwrite").parquet("../data/curated/fhvhv6")


yay! all done